In [31]:
# Dependencies
from bs4 import BeautifulSoup, SoupStrainer
import requests
import pandas as pd
from pprint import pprint
import re
import time

In [32]:
rent_obj = []
all_property = []

for page in range(20,29):
    link = 'https://www.apartments.com/Townhomes/ca/' + str(page) + '/'
    user_agent = 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36'
    headers = {'User-Agent': user_agent}
    response = requests.get(link, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
#     page 1-1
#     results = soup.find_all('article', class_='diamond placard')
#     page 1-1
#     results = soup.find_all('article', class_='platinum placard')
#     page 1-2
#     results = soup.find_all('article', class_='gold placard')
#     page 2-3
#     results = soup.find_all('article', class_='premiumPlus placard')
#     page 3-4
#     results = soup.find_all('article', class_='silver placard')
#     page 4-9
#     results = soup.find_all('article', class_='premium placard')
#     page 9-28
    results = soup.find_all('article', class_='prosumer placard')

    # Loop through returned results and get the hyperlink of the properties of the page
    for result in results:
        find_tag = result.find('a')['href']
        new_response = requests.get(find_tag, headers=headers)
        new_soup = BeautifulSoup(new_response.text, 'html.parser')
        new_results = new_soup.find('div', class_="mainWrapper")
        
        full_address = result.find('div', class_ = "location").text
        try:
            city = new_results.find('div', class_ = "propertyAddress").select('span')[0].text
        except:
            city = 'null'
        try:
            state = new_results.find('div', class_ = "propertyAddress").select('span')[1].text
        except:
            state = 'null'
        zip_code = new_results.find('div', class_ = "propertyAddress").select('span')[2].text
        street = new_results.find('h1', class_ = "propertyName").text.replace('\r\n                    ', "").strip()
        bedroom_container = new_results.find('div', class_="rentRollupContainer")
        house_details = new_results.find('table', class_="availabilityTable")
        try:
            bath_number = house_details.find('td', class_="baths").find('span', class_="shortText").text.replace('\r\n                    ', "").strip()
        except:
            bath_number = 'null'
        house_square_feet = house_details.find('td', class_="sqft").text
        square_feet = house_square_feet.split(" ")[0];
        lease_length = house_details.find('td', class_="leaseLength").text
        availability_check = house_details.find('td', class_="available").text.replace('\r\n                    ', "").strip()
        all_the_rents = bedroom_container.findAll('span', class_="rentRollup")

        bedroom_info = []
        for rent_block in all_the_rents:
            bedroom_type = rent_block.find('span', class_="shortText").text
            try:
                price = re.search('\$.* ', rent_block.text)
                price = price.group(0)[:-3]

            except:
                if "Call for Rent" in rent_block.text:
                    price = ""
                else:
                    price = re.search('\$.*', rent_block.text).group()[:-1]
            bedroom_info.append({
                    'bedroom_type': bedroom_type,
                    'price': price
                })
        rent_obj = {
                "City_Rent": city,
                "Type": "Townhomes",
                "State_Rent": state,
                "Zip_code": zip_code,
                "Bedroom_Info": bedroom_info,
                "Full_Address": full_address,
                "Rent_link": find_tag,
                "Bath_Info": bath_number,
                "Square_Feet_Data": house_square_feet,
                "Square_Feet": square_feet,
                "Lease_length": lease_length,
                "Availability_check": availability_check
            }
        for room_type in bedroom_info:
            rent_obj[room_type["bedroom_type"]] = room_type["price"];
        
        all_property.append(rent_obj)
        time.sleep(1)
print(all_property)

[{'City_Rent': '300 Myer Dr', 'Type': 'Townhomes', 'State_Rent': 'Chowchilla', 'Zip_code': 'CA', 'Bedroom_Info': [{'bedroom_type': '1 Bed', 'price': '$494'}], 'Full_Address': 'Chowchilla, CA 93610', 'Rent_link': 'https://www.apartments.com/300-myer-dr-chowchilla-ca/q7zwzcx/', 'Bath_Info': '1 BA', 'Square_Feet_Data': '', 'Square_Feet': '', 'Lease_length': '12 Month Lease', 'Availability_check': 'Available Now', '1 Bed': '$494'}, {'City_Rent': '2026 Barbara St', 'Type': 'Townhomes', 'State_Rent': 'Selma', 'Zip_code': 'CA', 'Bedroom_Info': [{'bedroom_type': '1 Bed', 'price': '$854'}], 'Full_Address': 'Selma, CA 93662', 'Rent_link': 'https://www.apartments.com/2026-barbara-st-selma-ca/kvgkheb/', 'Bath_Info': '1 BA', 'Square_Feet_Data': '', 'Square_Feet': '', 'Lease_length': '12 Month Lease', 'Availability_check': 'Available Now', '1 Bed': '$854'}, {'City_Rent': '2425 Shady Ln', 'Type': 'Townhomes', 'State_Rent': 'Anderson', 'Zip_code': 'CA', 'Bedroom_Info': [{'bedroom_type': '1 Bed', 'pric

In [33]:
rent_data = pd.DataFrame(all_property)
rent_data

,City_Rent,Type,State_Rent,Zip_code,Bedroom_Info,Full_Address,Rent_link,Bath_Info,Square_Feet_Data,Square_Feet,Lease_length,Availability_check,1 Bed,4 Beds,2 Beds,3 Beds,4+ Beds
0,300 Myer Dr,Townhomes,Chowchilla,CA,"[{'bedroom_type': '1 Bed', 'price': '$494'}]","Chowchilla, CA 93610",https://www.apartments.com/300-myer-dr-chowchi...,1 BA,,,12 Month Lease,Available Now,$494,NaN,NaN,NaN,NaN
1,2026 Barbara St,Townhomes,Selma,CA,"[{'bedroom_type': '1 Bed', 'price': '$854'}]","Selma, CA 93662",https://www.apartments.com/2026-barbara-st-sel...,1 BA,,,12 Month Lease,Available Now,$854,NaN,NaN,NaN,NaN
2,2425 Shady Ln,Townhomes,Anderson,CA,"[{'bedroom_type': '1 Bed', 'price': '$764'}]","Anderson, CA 96007",https://www.apartments.com/2425-shady-ln-ander...,1 BA,,,12 Month Lease,Available Now,$764,NaN,NaN,NaN,NaN
3,1565 Fair Oaks Dr,Townhomes,Anderson,CA,"[{'bedroom_type': '1 Bed', 'price': '$707'}]","Anderson, CA 96007",https://www.apartments.com/1565-fair-oaks-dr-a...,1 BA,,,12 Month Lease,Available Now,$707,NaN,NaN,NaN,NaN
4,73747 Raymond Way,Townhomes,Twentynine Palms,CA,"[{'bedroom_type': '1 Bed', 'price': '$653'}]","Twentynine Palms, CA 92277",https://www.apartments.com/73747-raymond-way-t...,1 BA,,,12 Month Lease,Available Now,$653,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220,Los Angeles,Townhomes,CA,90710,"[{'bedroom_type': '3 Beds', 'price': '$2,550'}]","26045 Frampton Ave, Los Angeles, CA 90710",https://www.apartments.com/26045-frampton-ave-...,2½ BAs,"1,620 Sq Ft","1,620",12 Month Lease,Available Now,NaN,NaN,NaN,"$2,550",NaN
221,Paso Robles,Townhomes,CA,93446,"[{'bedroom_type': '3 Beds', 'price': '$2,200'}]","Paso Robles, CA 93446",https://www.apartments.com/730-walnut-dr-paso-...,2 BAs,"1,100 Sq Ft","1,100",36 Month Lease,Jul 11,NaN,NaN,NaN,"$2,200",NaN
222,Berkeley,Townhomes,CA,94704,"[{'bedroom_type': '4 Beds', 'price': '$6,000'}]","Berkeley, CA 94704",https://www.apartments.com/1936-haste-st-berke...,2 BAs,"1,500 Sq Ft","1,500",12 Month Lease,Available Now,NaN,"$6,000",NaN,NaN,NaN
223,Rancho Cucamonga,Townhomes,CA,91730,"[{'bedroom_type': '2 Beds', 'price': '$2,150'}]","Rancho Cucamonga, CA 91730",https://www.apartments.com/8316-vineyard-ave-r...,2½ BAs,"1,021 Sq Ft","1,021",11 Month Lease,Available Now,NaN,NaN,"$2,150",NaN,NaN


In [34]:
# Export file as a CSV, without the Pandas index, but with the header
rent_data.to_csv("Output/Apartment.com/Q2-2020_Apartment.com_Townhomes_Data_page20-28.csv", index=False, header=True)